In [1]:
%cd ..
# %load_ext autoreload
# %autoreload 2
import sensor_lib as sl
import sensor_lib.data_analis as ds

/home/korbash/projects/torch_sensor_gpu/sashas_net


In [2]:
# %load_ext tensorboard

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join as jn
import yaml
import json

In [4]:
with open('params.yaml') as conf_file:
    config = yaml.safe_load(conf_file)
geo = config['env']['sen_geometry']
phys = config['env']['phys']

In [5]:
np.random.seed(config['random_seed'])
seeds = np.random.randint(0, 2**32, size=3)

# generation pressure_map

In [6]:
x = geo['x_len']
y = geo['y_len']
n_pic = config['dataset']['n_sampels']
n_gaus = config['env']['presure_profile']['n_gauses']
size_kof = config['env']['presure_profile']['size_kof']
vec_mat = tf.constant(sl.get_vec_mat(x, y), dtype=tf.float32)
vec_mat = tf.reshape(vec_mat, [-1, 2])
gaus_data = sl.gen_rand_cof(n_gaus * n_pic, x, y, size_kof,
                            seed=seeds[0])  # use random inside
gaus_data = tf.reshape(gaus_data, [n_pic, n_gaus, 5])
with open(config['env']['presure_profile']['g_param_path'], 'wb+') as f:
    np.save(f, gaus_data)

2022-06-11 14:39:29.021439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-11 14:39:29.881502: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1162 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0


In [7]:
dataset = tf.data.Dataset.from_tensor_slices(gaus_data)
batches = dataset.batch(config['gengaus']['batch_size'], drop_remainder=False)

In [ ]:
# tf.profiler.experimental.start('logs')
pictures=[]
for batch in batches:
    picture = sl.generate_pictures(batch, vec_mat)
    pictures.append(picture)
tf.profiler.experimental.stop()
pictures = tf.concat(pictures, axis=0)
pictures = tf.reshape(pictures, [n_pic, x, y])

In [ ]:
# writer = tf.summary.create_file_writer('logs')
# tf.summary.trace_on(graph=True, profiler=True)
# pic = sl.generate_multi_gaussian_flat(gaus_data[0], vec_mat)
# with writer.as_default():
#     tf.summary.trace_export(
#       name="multi_gaussian_trace",
#       step=0,
#       profiler_outdir='logs')

In [ ]:
pictures.shape

limit by hat fun

In [ ]:
norm = sl.round_fun([x, y], [x / 2, y / 2], lambda l: sl.hat(l, x / 2))
pictures = pictures * norm

In [ ]:
for i in range(10):
    plt.imshow(pictures[i])
    plt.show()

In [ ]:
with open(jn(config['dataset']['pic_path'], 'one_piece.npy'), 'wb+') as f:
    np.save(f, pictures)

# counting losses in fibers (input of nerual network)

In [6]:
    size = None
    max_possible_size = config['sim']['max_possible_size']
    test_size = config['sim']['test_size']
    batch_size = config['sim']['batch_size']
    n_del = geo['n_spl']
    mas = np.load(jn(config['dataset']['pic_path'],'one_piece.npy'), mmap_mode='r')
    if size == None:
        size = min(mas.shape[0], max_possible_size)
    if test_size == 'None':
        test_size = int(size / 10)
    mas = mas[0:size]
    mas = mas.astype('float32')

In [7]:
mas.shape

(100000, 64, 64)

In [8]:
config['sim']['test_mod'] = False
input, output, input_test, output_test = sl.sim_on_gpu(
    mas, test_size, batch_size, config=config,
    seed=seeds[1])  # use random inside

2022-06-11 19:04:48.689516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-11 19:04:57.868343: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30982 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:8b:00.0, compute capability: 7.0
2022-06-11 19:05:19.525551: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400
2022-06-11 19:05:32.383961: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to C

In [9]:
input.shape

(900000, 64, 4)

# defining nerual network for decoding and fitting it

In [10]:
print('input shape: ', input.shape, '\noutput shape: ', output.shape)

input shape:  (900000, 64, 4) 
output shape:  (900000, 64, 64)


In [11]:
tr = config['train']
model = sl.SensorNN5S_norm_deep(input.shape, output.shape)
model.build(input.shape)
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(tr['lerning_rate']),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
model_name = 'SensorNN5S_norm_deep'

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 4)]      0           []                               
                                                                                                  
 reshape (Reshape)              (None, 64, 4, 1)     0           ['input_1[0][0]']                
                                                                                                  
 Conv_1.1 (Conv2D)              (None, 64, 4, 8)     32          ['reshape[0][0]']                
                                                                                                  
 MaxPool_1.1 (MaxPooling2D)     (None, 32, 4, 8)     0           ['Conv_1.1[0][0]']               
                                                                                              

In [12]:
# model.fit(dataset_b, epochs=n_epochs, verbose=1)
tf.keras.utils.set_random_seed(seed=int(seeds[2]))  # use random
model.fit(input, output, epochs=tr['n_epochs'], verbose=1)

2022-06-11 19:06:48.754569: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14745600000 exceeds 10% of free system memory.
2022-06-11 19:07:10.129149: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14745600000 exceeds 10% of free system memory.


Epoch 1/50
28125/28125 [==============================] - 162s 6ms/step - loss: 0.2587 - accuracy: 0.0581
Epoch 2/50
28125/28125 [==============================] - 164s 6ms/step - loss: 0.2271 - accuracy: 0.0705
Epoch 3/50
28125/28125 [==============================] - 155s 6ms/step - loss: 0.2201 - accuracy: 0.0768
Epoch 4/50
28125/28125 [==============================] - 155s 6ms/step - loss: 0.2157 - accuracy: 0.0815
Epoch 5/50
28125/28125 [==============================] - 155s 5ms/step - loss: 0.2126 - accuracy: 0.0852
Epoch 6/50
28125/28125 [==============================] - 164s 6ms/step - loss: 0.2103 - accuracy: 0.0880
Epoch 7/50
28125/28125 [==============================] - 155s 6ms/step - loss: 0.2086 - accuracy: 0.0903
Epoch 8/50
28125/28125 [==============================] - 155s 6ms/step - loss: 0.2072 - accuracy: 0.0921
Epoch 9/50
28125/28125 [==============================] - 156s 6ms/step - loss: 0.2060 - accuracy: 0.0937
Epoch 10/50
28125/28125 [=====================

KeyboardInterrupt: 

In [13]:
model.save(jn(tr['models_path'], model_name + '.nn'))

INFO:tensorflow:Assets written to: models/SensorNN5S_norm_deep.nn/assets


INFO:tensorflow:Assets written to: models/SensorNN5S_norm_deep.nn/assets


# evoluate model on don't seen data

In [14]:
ev = config['evaluate']

In [15]:
input.shape

(900000, 64, 4)

In [16]:
model = tf.keras.models.load_model(jn(tr['models_path'], model_name + '.nn'))

In [17]:
test = model.evaluate(input_test, output_test)
train = model.evaluate(input, output)

313/313 [==============================] - 2s 4ms/step - loss: 0.2071 - accuracy: 0.0849


2022-06-11 20:04:49.876405: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14745600000 exceeds 10% of free system memory.
2022-06-11 20:05:14.009809: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 14745600000 exceeds 10% of free system memory.


28125/28125 [==============================] - 114s 4ms/step - loss: 0.1990 - accuracy: 0.1028


In [18]:
keys = ['loss','accuracy']
test_d = dict(zip(keys, test))
train_d = dict(zip(keys, train))
res = {'train': train_d, 'test': test_d}
with open(jn(ev['reports_path'], "summary.json"), "w") as f:
    json.dump(res, f)

In [17]:
predictions = model.predict(input_test)
predictions.shape

313/313 [==============================] - 1s 3ms/step


(10000, 64, 64)

# saving resalts

In [ ]:
with open(jn(ev['pred_path'], 'pred.npy'), 'wb') as f:
    np.save(f, predictions)
with open(jn(ev['pred_path'], 'true.npy'), 'wb') as f:
    np.save(f, output_test)

# look on resalts

In [ ]:
import sensor_lib.data_analis as ds

In [ ]:
# path_true= jn(ev['pred_path'], 'true.npy')
# path_pred= jn(ev['pred_path'], 'pred.npy')
# inter=(81000,90000)
# l=100000
# n_ga_mas=[15,30]
# n_fi_mas=[1,2,3,4,8,16]
# dic=ds.get_dic(n_ga_mas,n_fi_mas, path_true, path_pred, l, inter)
# df=pd.DataFrame(dic)

In [ ]:
with open(jn(ev['pred_path'], 'pred.npy'), 'rb') as f:
    predictions = np.load(f)
with open(jn(ev['pred_path'], 'true.npy'), 'rb') as f:
    output_test = np.load(f)

In [ ]:
N = 0
fig, axes = plt.subplots(1, 2, squeeze=True)
axes[0].imshow(predictions[N])
axes[0].set_title('pred')
axes[1].imshow(output_test[N])
axes[1].set_title('true')
fig.set_figwidth(5 * 2)
fig.set_figheight(5)
fig.savefig(jn(ev['reports_path'], 'result_sampel.png'))
plt.show()

In [ ]:
print('len= ', len(predictions))
N = 710 # number of exampel
fig = plt.imshow(predictions[N])
fig.savefig(jn(ev['report_path'],'result_sampel', 'pred.png'))
plt.show()
fig = plt.imshow(output_test[N])
plt.show()